# Combine Experiments

Notebook is used to craft individual experiment files from other experiment files (e.g. Gear D1 experiment from vehicle speed experiments).

## Imports

In [1]:
from pathlib import Path
from datetime import date
import os
import subprocess
import random
from copy import deepcopy
from typing import List
from revcan.reverse_engineering.models.experiment import Experiment, Extern_Alphanumeric_Signal, Extern_Signal

## Set Experiment Names manually

In [ ]:
car_model_file_name = "model_vin_date-created.json"
car_model_dir = "../../data/car_metadata"
experiment_folder = "../../data/experiments/car/speed-drive/2025-02-24_10_02_04" # TODO: Set folder of existing experiment files
ground_truth_values=["10kmh","20kmh","30kmh","40kmh"] # TODO: Set file names from which data is taken
number_of_values_per_gt = 2 # TODO: Set how many values from each file should be taken for the new experiment file

car_model_file = f"{car_model_dir}/{car_model_file_name}"
car_model_file_path = os.path.abspath(car_model_file)
Path(experiment_folder).mkdir(parents=True, exist_ok=True)

## [Optional] Display Experiment Metadata

In [ ]:
# Display Experiment
for ground_truth_value in ground_truth_values:
    experiment_file = os.path.join(experiment_folder, f"{ground_truth_value}.json")
    !python ../scripts_for_doip_new/display_experiment_metadata.py --experiment_file_path "{experiment_file}"

## Load Experiments

Load all relevant experiments needed for this analysis

In [4]:
experiment_files = {}
experiments = {}

for ground_truth_value in ground_truth_values:
    experiment_files[ground_truth_value] = os.path.join(experiment_folder, f"{ground_truth_value}.json")
    experiments[ground_truth_value] = Experiment.load(experiment_files[ground_truth_value])

Concat all experiments

In [ ]:
combined_experiment_file_name = "D1" # TODO: Set new file name
combined_experiment_file_path = os.path.join(experiment_folder, f"{combined_experiment_file_name}.json")
combined_experiment_name = f"Gear selected D1 (from Drive Experiments)" # TODO: Set new experiment name
combined_experiment_description = f"Manually crafted experiment for gear D1 for gear selected signal discovery. Experiment is created from Speed Drive measurements for speeds {ground_truth_values}" # TODO: Set new experiment description
signal_selection ="all"
print(f"Experiment file path: {combined_experiment_file_path}")
!python ../scripts_for_doip_new/04_create_experiment.py --car_model_file_path {car_model_file_path} --experiment_file_path '{combined_experiment_file_path}' --experiment_name '{combined_experiment_name}' --experiment_description '{combined_experiment_description}' --signal_selection {signal_selection}

#experiment_file = os.path.join(experiment_folder, f"{combined_experiment_file_name}.json")
combined_experiment = Experiment.load(combined_experiment_file_path)

for ground_truth_value in ground_truth_values:
    combined_experiment.experiment_runtime_seconds += (experiments[ground_truth_value].experiment_runtime_seconds * number_of_values_per_gt / len(experiments[ground_truth_value].measurements[-1].values))
    for measurement in experiments[ground_truth_value].measurements:
        for x in combined_experiment.measurements:
            if x.serverid == measurement.serverid:
                if x.did == measurement.did:
                    limited_values = random.sample(measurement.values, min(number_of_values_per_gt, len(measurement.values)))
                    x.values.extend(limited_values)
                    break


Add ground truth value

In [ ]:
ground_truth = "D1"
combined_experiment.external_alphanumeric_measurements.append(Extern_Alphanumeric_Signal( name = "Gear", id = 1, values = []))
values:List[str] = []
for value in combined_experiment.measurements[0].values:
    values.append(ground_truth)
combined_experiment.external_alphanumeric_measurements[0].name = "Gear"
combined_experiment.external_alphanumeric_measurements[0].values = deepcopy(values)
print(f"Combined Experiment: {len(combined_experiment.external_alphanumeric_measurements[0].values)} ground truth values")

In [ ]:
combined_experiment_file = os.path.join(experiment_folder, f"{combined_experiment_file_name}.json")
combined_experiment.save(f"{combined_experiment_file}")
!python ../scripts_for_doip_new/display_experiment_metadata.py --experiment_file_path "{combined_experiment_file}"